In [ ]:
import pandas as pd
import json

In [ ]:
with open("clusters.json", "r") as f:
    clusters = json.load(f)

In [ ]:
df = pd.read_csv("../Dataset/dataset.csv")
df.head()

In [ ]:
from nltk import word_tokenize
import string
def clusterfy(sentence):
    #cleaning the sentence
    if not isinstance(sentence, str):
        return []
    for p in string.punctuation:
        sentence = sentence.replace(p, " ")
    sentence = sentence.replace("  ", " ").lower()
    
    
    tokens = word_tokenize(sentence)
    for i,token in enumerate(tokens):
            if token in clusters["clusters"]:
                cluster_id = clusters["clusters"][token]
                tokens[i] = clusters["replace"][str(cluster_id)]
                
    return tokens

## Creating the bag of words with the clusters

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import  RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier

In [ ]:
bag_of_words = []
for index, row in df.iterrows():
    new_lyrics = clusterfy(row["lyrics"])
    new_title = clusterfy(row["title"])
    bag_of_words.append(new_lyrics + new_title)

## Running classifiers

In [ ]:
vectorizer = CountVectorizer(
    min_df=10,
    lowercase=True,
    strip_accents="unicode",
    stop_words="english",
    ngram_range=(2,2)
)
vectors = vectorizer.fit_transform(bag_of_words)

In [ ]:
sss = ShuffleSplit(n_splits=1, test_size=0.3)

In [ ]:
classifiers = {
    "Naive Bayes": MultinomialNB(alpha=1),
    "SGCD" : SGDClassifier(),
    "Random Forest": RandomForestClassifier(),
    "KNeighborsClassifier" : KNeighborsClassifier(3),
    "DecisionTreeClassifier": DecisionTreeClassifier(max_depth=5),
}

In [ ]:
labels = np.array(styles.tolist())
for train_index, test_index in sss.split(vectors, labels):
    X_train, X_test = vectors[train_index], vectors[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    categories = np.unique(labels[test_index]).tolist()
    
    # instanciamos o classificador
    for classifier_name, classifier in classifiers.items():
        print(f"=========== {classifier_name} =========")
        clf = classifier
        # colocamos os nossos dados de treino
        clf.fit(X_train, y_train)
        # medimos sua acurácia com os dados de teste
        predictions = clf.predict(X_test)
        print(classification_report(y_test, predictions, target_names=categories))
        print(f"SCORE: {clf.score(X_test, y_test)}")
        print('-'*100)